In [ ]:
cd /your/direcotry/

In [ ]:
import csv
import pyfaidx
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
import random
import warnings
import pybedtools
import re
import numpy as np

### Extract the 10 bp flanking the start and end poisition of eccDNA cooridnates

In [ ]:
hg38_fasta = pyfaidx.Fasta("hg38.fa")

In [ ]:
# Read in the CSV file
df = pd.read_csv('eccDNA_coordinates.csv')

# Define a function to extract the 21 bp sequence 
def extract_seq(chr_name, pos):
    start = pos - 11
    end = pos + 10
    return str(hg38_fasta[chr_name][start:end])

# Start sequences
with open('eccDNA_start_junction.fa', 'w') as out_file:
    # Loop through the dataframe and extract sequences
    for i, row in df.iterrows():
        # Extract start sequence
        start_seq = extract_seq(row['chr'], row['start'])
        out_file.write(f'>start_{i}_{row["chr"]}_{row["start"]}\n{start_seq}\n')
        
# End sequences
with open('eccDNA_end_junction.fa', 'w') as out_file:
    # Loop through the dataframe and extract sequences
    for i, row in df.iterrows():        
        # Extract end sequence
        end_seq = extract_seq(row['chr'], row['end'])
        out_file.write(f'>end_{i}_{row["chr"]}_{row["end"]}\n{end_seq}\n')

### Identify microhomology at the start and end sequences of eccDNA junctions

In [ ]:
# Perform the local sequence alignment using the Smith-Waterman algorithm.
# Return the length of the longest consective identical sequence.
def local_sequence_alignment(seq1, seq2):
    m = len(seq1)
    n = len(seq2)

    # Initialize the score matrix and traceback matrix
    score_matrix = [[0] * (n + 1) for _ in range(m + 1)]
    traceback_matrix = [[0] * (n + 1) for _ in range(m + 1)]

    # Variables to keep track of the highest score and its position in the matrix
    max_score = 0
    max_i = 0
    max_j = 0

    # Fill in the score and traceback matrices
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = score_matrix[i - 1][j - 1] + (seq1[i - 1] == seq2[j - 1])
            delete = score_matrix[i - 1][j]
            insert = score_matrix[i][j - 1]
            score = max(0, match, delete, insert)
            score_matrix[i][j] = score

            if score == 0:
                traceback_matrix[i][j] = 0
            elif score == match:
                traceback_matrix[i][j] = 1
            elif score == delete:
                traceback_matrix[i][j] = 2
            else:
                traceback_matrix[i][j] = 3

            # Update the maximum score and its position
            if score > max_score:
                max_score = score
                max_i = i
                max_j = j

    # Traceback to find the alignment
    align_seq1 = ""
    align_seq2 = ""

    i = max_i
    j = max_j

    identical_length = 0
    current_length = 0

    while i > 0 and j > 0 and score_matrix[i][j] > 0:
        if traceback_matrix[i][j] == 1:
            align_seq1 = seq1[i - 1] + align_seq1
            align_seq2 = seq2[j - 1] + align_seq2
            i -= 1
            j -= 1
            current_length += 1
        elif traceback_matrix[i][j] == 2:
            align_seq1 = seq1[i - 1] + align_seq1
            align_seq2 = '-' + align_seq2
            i -= 1
            current_length = 0
        else:
            align_seq1 = '-' + align_seq1
            align_seq2 = seq2[j - 1] + align_seq2
            j -= 1
            current_length = 0

        if current_length > identical_length:
            identical_length = current_length

    return identical_length

In [ ]:
# Define a function to extract the 20 bp sequence before the start position and after the end position of each eccDNA
def extract_start_seq(chr_name, pos):
    start = pos
    end = pos + 20
    return str(hg38_fasta[chr_name][start:end])

def extract_end_seq(chr_name, pos):
    start = pos - 20
    end = pos
    return str(hg38_fasta[chr_name][start:end])

In [ ]:
# Read in the CSV file
df = pd.read_csv('eccDNA_coordinates.csv')

homology_dict = {}
for i, row in df.iterrows():
    start_seq = extract_start_seq(row['chr'], row['start'])
    end_seq = extract_end_seq(row['chr'], row['end'])
    identical_length = local_sequence_alignment(start_seq, end_seq)
    homology_dict[f"{row['chr']}_{row['start']}_{row['end']}"] = identical_length
    
homology_df = pd.DataFrame.from_dict(homology_dict, orient ='index')
homology_df= homology_df.rename(columns={0: "homology_length"})
homology_df.to_csv('eccDNA_microhomology_length.csv')
homology_cnt = homology_df['homology_length'][homology_df['homology_length'] > 2].count()
pct_homology = homology_cnt/len(homology_df)*100
print('Pct of eccDNA having microhomology:'+ f'{pct_homology}')
homology_df.head()

In [ ]:
# Plot histogram
sns.histplot(homology_df, x="homology_length")

### Generate a random eccDNA dataset that has the same length and chromosome distribution as the actual dataset

In [ ]:
#hide warnings
warnings.filterwarnings('ignore')

# Read in the CSV file
df = pd.read_csv('eccDNA_coordinates.csv',header=0)

# Load the hg38 genome reference
genome = pyfaidx.Fasta("hg38.fa")

# Create an empty dataframe to store the random coordinates and sequences
rand_df = pd.DataFrame(columns=['chr', 'start', 'end'])

num_iter = 100
for j in range(num_iter):
    print(f'this is iteration {j}')
# Iterate over each chromosome in the input set
    for chrom, chrom_df in df.groupby('chr'):
        # Calculate the proportion of coordinates from this chromosome
        #print(f'working on {chrom}')
        chrom_prop = len(chrom_df) / len(df)

        # Calculate the total length of the chromosome
        chrom_len = len(genome[chrom])

        # Calculate the number of sequences to generate for this chromosome
        n_rand = len(chrom_df)

        # Generate random coordinates and sequences
        for i in range(n_rand):
            # Generate random coordinates
            rand_start = random.randint(0, chrom_len)
            seq_lengths = chrom_df.iloc[i,2] - chrom_df.iloc[i,1]      
            rand_end = rand_start + seq_lengths
            if rand_end > chrom_len:
                continue

            # Add the random sequence to the dataframe
            rand_df = rand_df.append({'chr': chrom, 'start': rand_start, 'end': rand_end}, ignore_index=True)

rand_df = rand_df.drop_duplicates()
# Output the results to a new CSV file
rand_df.to_csv('random_set_eccDNA.csv', index=False)

### Extract the 10 bp flanking the start and end poisition of the ramdom set

In [ ]:
# Define a function to extract the 21 bp sequence
def extract_seq(chr_name, pos):
    start = pos - 11
    end = pos + 10
    return str(genome[chr_name][start:end])

# Open the output fasta file for writing
with open('random_start_junction.fa', 'w') as out_file:
    # Loop through the dataframe and extract sequences
    for i, row in rand_df.iterrows():
        # Extract start sequence
        start_seq = extract_seq(row['chr'], row['start'])
        out_file.write(f'>start_{i}_{row["chr"]}_{row["start"]}\n{start_seq}\n')
        
# Open the output fasta file for writing
with open('random_end_junction.fa', 'w') as out_file:
    # Loop through the dataframe and extract sequences
    for i, row in rand_df.iterrows():        
        # Extract end sequence
        end_seq = extract_seq(row['chr'], row['end'])
        out_file.write(f'>end_{i}_{row["chr"]}_{row["end"]}\n{end_seq}\n')

### Calculate the percentage of eccDNA that contains at least one gene exon

In [ ]:
#Define a function
def extract_gene_name(cell):
    return cell.split('gene_name')[-1].replace('"', '').replace(';', '').strip()

# load gene annotation GTF file
gtf_file = 'hg38.ncbiRefSeq.gtf.gz'
gtf = pybedtools.BedTool(gtf_file)
gtf_df = gtf.to_dataframe(names=['chr', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])

# extract exon features from GTF
exon_df = gtf_df[gtf_df['feature'] == 'exon']

# load genome coordinates CSV file
coord_file = 'eccDNA_coordinates.csv'
coord_df = pd.read_csv(coord_file)

exon_lost_df = pd.DataFrame()
num_exon_coords = 0
for index, row in coord_df.iterrows():
    chr_name = row['chr']
    start_pos = row['start']
    end_pos = row['end']
    
    # subset gene annotation dataframe for the chromosome
    chr_gtf_df = exon_df[exon_df['chr'] == chr_name]
    
    # check if coordinate overlaps with exon
    overlapping_exons = chr_gtf_df[(chr_gtf_df['end'] <= end_pos) & (chr_gtf_df['start'] >= start_pos)]
    if len(overlapping_exons) > 0:
        num_exon_coords += 1
        exon_lost_df = pd.concat([exon_lost_df, overlapping_exons])
                
# calculate percentage of coordinates that contain at least one gene exon
percent_exon_coords = num_exon_coords / len(coord_df) * 100
num_eccDNA = len(coord_df)

exon_lost_df= exon_lost_df.drop_duplicates()
exon_lost_df["attribute"] = exon_lost_df["attribute"].map(lambda x : extract_gene_name(x))
exon_lost_df.to_csv('genes_losing_at_least_one_exon_due_to_eccDNA.csv')
print(f"{num_eccDNA} eccDNA in total.")
print(f"{num_exon_coords} eccDNA contain at least one gene exon.")
print(f"{percent_exon_coords}% of eccDNA contain at least one gene exon.")
exon_lost_df.head(3)